# es 1

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib as ptlt
import plotly.express as px
%matplotlib inline
import plotly.graph_objs as go

## lettura da csv

In [47]:
df_solar_panel=pd.read_csv("solar_panels.csv",sep=";")
df_solar_panel["DaySum"]=pd.to_numeric(df_solar_panel["DaySum"])

In [48]:
df_solar_panel["Date"]=pd.to_datetime(df_solar_panel["Date"]).dt.date

In [49]:
df_kwh_giorno=pd.DataFrame(df_solar_panel.groupby("Date").max("DaySum")["DaySum"]/1000)

In [50]:
len(df_kwh_giorno)

365

In [139]:
px.bar(df_kwh_giorno,
       y="DaySum",
       template='plotly_white',
       labels={"DaySum":"Somma kW/h","index":"Tempo (giorni)"},
       title="Produzione energetica totale giornaliera")

In [52]:
df_kwh_giorno["Date"]=pd.to_datetime(df_kwh_giorno.index)

In [53]:
df_kwh_giorno

,DaySum,Date
Date,,
2015-01-01,12.204,2015-01-01
2015-01-02,11.447,2015-01-02
2015-01-03,18.222,2015-01-03
2015-01-04,37.805,2015-01-04
2015-01-05,5.912,2015-01-05
...,...,...
2015-12-27,1.727,2015-12-27
2015-12-28,2.495,2015-12-28
2015-12-29,3.985,2015-12-29


In [54]:
df_kwh_settimana=pd.DataFrame(df_kwh_giorno["Date"].dt.isocalendar())
df_kwh_settimana["DaySum"]=df_kwh_giorno["DaySum"]
df_kwh_settimana=pd.DataFrame(df_kwh_settimana.groupby("week").sum("DaySum")["DaySum"])


In [138]:
fig = px.bar(df_kwh_settimana,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","week":"Settimane"},
             color_discrete_sequence=["green"],
             title="Produzione energetica totale settimanale")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in df_kwh_settimana.index]
    )
)
fig.show()

In [56]:
df_kwh_mese = df_kwh_giorno.groupby(df_kwh_giorno["Date"].dt.month).sum("DaySum")

In [57]:
fig = px.bar(df_kwh_mese,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","Date":"Mesi"},
             color_discrete_sequence=["orange"],
             title="Produzione energetica totale mensile")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in df_kwh_mese.index],
        ticktext = ["Gen","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    )
)
fig.show()

In [58]:
df_kwh_stagioni = pd.DataFrame(index=["Primavera","Estate","Autunno","Inverno"],columns=["DaySum"]).fillna(0)
df_kwh_stagioni
inizio_primavera = pd.to_datetime("2015-03-21")
inizio_estate = pd.to_datetime("2015-06-21")
inizio_autunno = pd.to_datetime("2015-09-22")
inizio_inverno = pd.to_datetime("2015-12-22")



In [59]:
kwh_totale = sum(df_kwh_giorno.DaySum)

In [60]:
df_kwh_stagioni.loc["Primavera","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.Date>=inizio_primavera) & (df_kwh_giorno.Date<inizio_estate)].DaySum)
df_kwh_stagioni.loc["Estate","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.Date>=inizio_estate) & (df_kwh_giorno.Date<inizio_autunno)].DaySum)
df_kwh_stagioni.loc["Autunno","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.Date>=inizio_autunno) & (df_kwh_giorno.Date<inizio_inverno)].DaySum)
df_kwh_stagioni.loc["Inverno","DaySum"] = kwh_totale - sum(df_kwh_stagioni.DaySum)

In [137]:
fig = px.bar(df_kwh_stagioni,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","index":"Stagioni"},
             color_discrete_sequence=["purple"],
             title="Produzione energetica totale stagionale",
             width=700,
             height=400,)
for data in fig.data:
    data["width"] =0.5
fig.show()

In [135]:
fig = px.bar(df_kwh_stagioni.transpose(),
             template='plotly_white',
             labels={"value":"kW/H","index":"", "variable":"Stagioni"},
             color_discrete_sequence= px.colors.qualitative.Plotly,
             title="Produzione energetica totale",
             width=600,
             height=500,
             barmode = "stack")
fig.update_xaxes(showticklabels=False)
for data in fig.data:
    data["width"] =0.4
fig.show()

In [131]:
px.pie(df_kwh_stagioni,  
        values='DaySum', 
        names=df_kwh_stagioni.index,
        title=f"Influenza delle stagioni sull'energia consumata ({int(kwh_totale)} kw/H)",
        width=600,
        height=500,)

In [64]:
risparmio = kwh_totale * 0.35
risparmio = round(risparmio, 2)
risparmio

2821.73

In [65]:
guadagno = (kwh_totale - 2700) * 0.05
guadagno = round(guadagno, 2)
guadagno

268.1

In [140]:
fabb_med_giornaliero = 2700 /365
fig = px.bar(df_kwh_giorno,
       y="DaySum",
       template='plotly_white',
       labels={"DaySum":"Somma kW/h","index":"Tempo (giorni)"},
       title="Produzione energetica totale giornaliera")
fig.add_hline(y=fabb_med_giornaliero,opacity=0.5)
fig.show()

In [67]:
df_guadagno_giornaliero = df_kwh_giorno.copy()
df_guadagno_giornaliero["Energia_Guadagnata"] = (df_guadagno_giornaliero["DaySum"] - fabb_med_giornaliero)

In [90]:
df_guadagno_giornaliero["Guadagno"]=0
df_guadagno_giornaliero.loc[df_guadagno_giornaliero["Energia_Guadagnata"] > 0, "Guadagno"] = df_guadagno_giornaliero["Energia_Guadagnata"]*0.05

df_guadagno_giornaliero["Spesa"]=0
df_guadagno_giornaliero.loc[df_guadagno_giornaliero["Energia_Guadagnata"] < 0, "Spesa"] = df_guadagno_giornaliero["Energia_Guadagnata"]*-0.35

df_guadagno_giornaliero["Ricavo"] = df_guadagno_giornaliero["Guadagno"] - df_guadagno_giornaliero["Spesa"]


df_guadagno_giornaliero["Color"] = ["Green" if x>0 else "Red" for x in df_guadagno_giornaliero["Ricavo"]]
df_guadagno_giornaliero

,DaySum,Date,Energia_Guadagnata,Guadagno,Spesa,Ricavo,Color
Date,,,,,,,
2015-01-01,12.204,2015-01-01,4.80674,0.240337,0.000000,0.240337,Green
2015-01-02,11.447,2015-01-02,4.04974,0.202487,0.000000,0.202487,Green
2015-01-03,18.222,2015-01-03,10.82474,0.541237,0.000000,0.541237,Green
2015-01-04,37.805,2015-01-04,30.40774,1.520387,0.000000,1.520387,Green
2015-01-05,5.912,2015-01-05,-1.48526,0.000000,0.519841,-0.519841,Red
...,...,...,...,...,...,...,...
2015-12-27,1.727,2015-12-27,-5.67026,0.000000,1.984591,-1.984591,Red
2015-12-28,2.495,2015-12-28,-4.90226,0.000000,1.715791,-1.715791,Red
2015-12-29,3.985,2015-12-29,-3.41226,0.000000,1.194291,-1.194291,Red


In [99]:
num_giorni_guadagno = len(df_guadagno_giornaliero[df_guadagno_giornaliero["Guadagno"] > 0])
num_giorni_guadagno

308

In [121]:
df_guadagno_giornaliero_cumulativo = pd.DataFrame(df_guadagno_giornaliero["Guadagno"].cumsum())
fig = px.line(df_guadagno_giornaliero_cumulativo,
        template='plotly_white',
        labels={"value":"Guadagno (€)","Date":"Tempo"},
        title="Guadagno cumulativo giornaliero",
        width=1000,
        height=700)

fig.update_layout(showlegend=False)
fig.show()

In [116]:
fig = px.bar(df_guadagno_giornaliero, y = "Ricavo", 
     template='plotly_white',
       labels={"index":"Tempo (giorni)","Ricavo":"Ricavo (€)"},
       title="Ricavo giornaliero",
       color = "Color",color_discrete_sequence=df_guadagno_giornaliero.Color.unique(),
       width=1400,
       height=450)

fig.update_layout(showlegend=False)
fig.show()